<h1> One file to gather all raw data and convert to cleaned, useable, csv files </h1>

<h1> 1) Weightlifting Dataset </h1>

Based on data from [Summer and Winter Olympics, 1896-2022](https://www.kaggle.com/datasets/piterfm/olympic-games-medals-19862018/).
Handled by Nicholas Stach.

<h2> Description </h2>

This dataset is on Olympic games for both the Summer and Winter Olympics, from 1896 to 2022. The data contains all events of each Olympic game and the medal results, i.e., which countries won a bronze, silver, or gold medal for each event. This data was harvested from the official [Olympics website](https://olympics.com/). For each Olympic games in the dataset, it is broken down by event, categories within the event (by gender), and the medal won by the athlete or team, and their associated country. For this project, the dataset will be trimmed down to focus exclusively on the weightlifting events from 1896-2020 (the most recent summer games in the dataset).

### Data Quirks:

1) Weight classes have changed numerous times over the course of Olympic game history, and are thus not uniform!
2) There was no weightlifting event for the 2022 Beijing Games. Weightlifting is a summer sport and the most recent game recorded was a winter game.

In [53]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import data & clean
# From: https://www.kaggle.com/datasets/piterfm/olympic-games-medals-19862018/
filePath = '../Raw Data/Summer-Winter Olympics/'
fileName = 'olympic_medals.csv'
colHeads = ["discipline_title", "oly_game", "event_title", "event_gender", "medal_type", "participant_type", 
            "particpant_title", "athlete_url", "athlete_full_name", "country_name", "country_code_2_letter", "country_code"]        # column names
allSports = pd.read_csv(filePath+fileName, skiprows=1, names=colHeads)                                                              # read in all data (every sport)

# Focus exclusively on weightlifting
weightDF = allSports[allSports['discipline_title'] == 'Weightlifting']                                                              # all weightlifting medalists, by game, 1896-2022

# Trim data of labels that are redundant for our purposes
weightDF = weightDF.drop(columns=["discipline_title", "participant_type", "particpant_title", "athlete_url", 
                                  "country_code_2_letter"], axis=1)

# Add explicit year column (to help in data analysis later)
years = pd.Series(weightDF["oly_game"].str[-4:], name='year', dtype='int')                                                          # get year for each row (last four characters of 'oly_game')
weightDF = pd.concat([weightDF, years], axis=1)                                                                                     # concatenate, column-wise
# print("Cleaned data: ")
# display(weightDF)

# Create file for cleaned data
weightDF.to_csv("weightlifting medals.csv", index=False)

df1 = pd.read_csv("./weightlifting medals.csv")
print("New file for cleaned data: ")
display(df1)

New file for cleaned data: 


,oly_game,event_title,event_gender,medal_type,athlete_full_name,country_name,country_code,year
0,tokyo-2020,Men's 61kg,Men,SILVER,Eko Yuli IRAWAN,Indonesia,INA,2020
1,tokyo-2020,Men's 61kg,Men,GOLD,Fabin LI,People's Republic of China,CHN,2020
2,tokyo-2020,Men's 61kg,Men,BRONZE,Igor SON,Kazakhstan,KAZ,2020
3,tokyo-2020,Women's 55kg,Women,SILVER,Qiuyun LIAO,People's Republic of China,CHN,2020
4,tokyo-2020,Women's 55kg,Women,BRONZE,Zulfiya CHINSHANLO,Kazakhstan,KAZ,2020
...,...,...,...,...,...,...,...,...
674,athens-1896,heavyweight - one hand lift men,Men,SILVER,Viggo JENSEN,Denmark,DEN,1896
675,athens-1896,heavyweight - one hand lift men,Men,BRONZE,Alexandros Nikolopoulos,Greece,GRE,1896
676,athens-1896,heavyweight - two hand lift men,Men,GOLD,Viggo JENSEN,Denmark,DEN,1896
677,athens-1896,heavyweight - two hand lift men,Men,SILVER,Launceston ELLIOT,Great Britain,GBR,1896


<h1> 2) Cost of Healthy Diet Dataset </h1>

Based on this [dataset](https://databank.worldbank.org/source/food-prices-for-nutrition).
Handled by Thibault Eelen.


<h2> Description </h2>
Food Prices for Nutrition provides indicators on the cost and affordability of healthy diets in each country, showing the population’s physical and economic access to sufficient quantities of locally available items for an active and healthy life over 5 years (2017-2021). It also provides indicators on the cost and affordability of an energy-sufficient diet and a nutrient-adequate diet. It also has a column with each country's population each year. 


### Data Quirks:

1) Not every country has values in every column, for every year. This will require some handling of NaNs and adjustment of averages.
2) Does not share all country names and country codes with Weightlifting data, will need extra care when comparing by country across datasets.

### NOTE:
    
1) 'millions_healthy_diet'  -> millions who cannot afford a healthy diet at 52% of income
2) 'millions_nutrient'      -> millions who cannot afford nutrient adequacy at 52% of income
3) 'millions_calories'      -> millons who cannot afford sufficient calories at 52% of income
4) 'percent_healthy_diet'   -> percent of pop who cannot afford healthy diet
5) 'percent_nutrient'       -> percent of pop who cannot afford nutrient adequacy
6) 'percent_calories'       -> percent of pop who cannot afford sufficient calories
7) 'affordability_energy'   -> affordability of energy sufficient diet (ratio of cost to food expenditures)
8) 'affordability_nutrient' -> affordability of nutrient adequate diet (ratio of cost to food expenditures)
9) 'affordability_healthy_diet' -> affordability of healthy diet (ratio of cost to food expenditures)


In [54]:
# Import data & clean
# From: https://databank.worldbank.org/source/food-prices-for-nutrition
path2file = '../Raw Data/Healthy Diet and Populations/'
fn = 'cost of healthy diet.csv'
colHeads = ["Classification Name", "Classification Code", "country_name", "country_code", "year", "Time Code", "population",
            "millions_healthy_diet", "millions_nutrient", "millions_calories", "percent_healthy_diet", "percent_nutrient",
            "percent_calories", "affordability_energy", "affordability_nutrient", "affordability_healthy_diet" ]
DF = pd.read_csv(path2file+fn, skiprows=1, names=colHeads, encoding='iso-8859-1') 


DFDiet = DF.drop(columns=["Classification Name", "Classification Code", "Time Code", ])  # remove unneeded columns
DFDiet = DFDiet.drop([930, 931, 932, 933, 934])                            # delete update and author info at bottom of file
DFDiet.replace('..', np.nan, inplace=True)                                 # turn invalid data ('..') into NaN for easy exclusion during analysis

columnTypes = {'country_name': str,
               'country_code': str,
               'year': float, 
               'population': float,
               'millions_healthy_diet': float,
               'millions_nutrient': float,
               'millions_calories': float,
               'percent_healthy_diet': float,
               'percent_nutrient': float,
               'percent_calories': float,
               'affordability_energy': float,
               'affordability_nutrient': float,
               'affordability_healthy_diet': float}                
DFDiet = DFDiet.astype(columnTypes)                                     # convert every column to expected dtype

# print("Cleaned data: ")
# display(DFDiet)

# Create file for cleaned data
DFDiet.to_csv("healthy diet.csv", index=False)
df2 = pd.read_csv("./healthy diet.csv")
print("New file for cleaned data: ")
display(df2)

New file for cleaned data: 


,country_name,country_code,year,population,millions_healthy_diet,millions_nutrient,millions_calories,percent_healthy_diet,percent_nutrient,percent_calories,affordability_energy,affordability_nutrient,affordability_healthy_diet
0,Albania,ALB,2017.0,2873457.0,0.9,0.3,0.0,31.3,8.9,0.0,0.078,0.266,0.425
1,Albania,ALB,2018.0,2866376.0,0.7,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN
2,Albania,ALB,2019.0,2854191.0,0.6,NaN,NaN,22.2,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,2020.0,2837849.0,0.6,NaN,NaN,19.9,NaN,NaN,NaN,NaN,NaN
4,Albania,ALB,2021.0,2811666.0,0.4,NaN,NaN,15.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,Zimbabwe,ZWE,2017.0,14751101.0,10.0,1.6,0.0,67.8,10.6,0.0,0.106,0.314,0.973
926,Zimbabwe,ZWE,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
927,Zimbabwe,ZWE,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
928,Zimbabwe,ZWE,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h1> 3) GDP by country, 1990 - 2018 </h1>

Based on this [dataset](https://www.kaggle.com/datasets/nitishabharathi/gdp-per-capita-all-countries). Handled by Pronob Sarker.


<h2> Description </h2>
Gross domestic product of all countries worldwide, from 1990 to 2018. Gross domestic product is an estimated size of a country's economy and growth rate. GDPs in dataset are based on Purchasing Power Parity (PPP).


### Data Quirks:

1) Some countries had no data for all years and where excluded from the cleaned data. Additionally, some countries have data for only some of the years included in the dataset.



In [55]:
# Load the data from the CSV file
file_path = '../Raw data/GDP(1990-2018).csv'
data = pd.read_csv(file_path)


# GDP data starts from the third column onward
gdp_columns = data.columns[2:]

# Remove rows where all GDP values across the years are NaN
data = data.dropna(subset=gdp_columns, how='all')

data.drop(columns=["2019"], inplace=True, axis=1)               # drop '2019' column (has no data)


# print("Cleaned data:")
# display(data)

# Convert cleaned data to csv
data.to_csv("country GDP.csv", index=False)
gdpDF = pd.read_csv("country GDP.csv")

print("New file from cleaned data: ")
display(gdpDF)

New file from cleaned data: 


,Country,Country Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,24101.109430,25870.755940,26533.343900,27430.752400,28656.520210,28648.990020,28499.089430,30215.949230,...,34693.086750,33732.847450,35492.618490,35498.982090,37419.892820,38223.372260,38249.054870,38390.271650,39454.629830,NaN
1,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1454.663015,1637.377987,1626.764793,1806.763930,1874.765634,1897.525938,1886.692977,1896.992520,1934.636754,1955.006208
2,Angola,AGO,3089.683369,3120.356148,2908.160798,2190.768160,2195.532289,2496.199493,2794.896906,2953.342709,...,6092.783245,6230.297028,6346.395122,6772.528333,6980.423038,7199.245478,7096.600615,6756.935074,6650.584940,6452.355165
3,Albania,ALB,2549.473022,1909.114038,1823.307673,2057.449657,2289.873135,2665.764906,2980.066288,2717.362124,...,8814.810856,9628.025783,10207.752350,10526.235450,10571.010650,11259.225890,11662.030480,11868.178970,12930.140030,13364.155400
4,Arab World,ARB,6808.206995,6872.273195,7255.328362,7458.647059,7645.682856,7774.207360,8094.149842,8397.515692,...,13640.846780,14127.778020,14518.827450,15423.465390,15824.780110,16153.244860,16501.792590,16935.383300,17099.889390,17570.137600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,Kosovo,XKX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7506.898620,7782.875107,8222.285712,8547.652384,8903.763429,9194.741819,9781.019679,10208.857900,10756.663460,11348.363450
239,"Yemen, Rep.",YEM,2223.028771,2325.263661,2443.920401,2472.188808,2569.648739,2657.813447,2730.145147,2829.866193,...,4221.413776,4472.726448,3876.302619,3935.167105,4084.882967,4045.500031,3320.110156,2827.691023,2645.308383,2575.126385
240,South Africa,ZAF,6424.502215,6414.057923,6261.716184,6331.777580,6520.866604,6719.583315,7000.801619,7179.806651,...,11416.001690,11728.160490,12179.174190,12488.215570,12815.727070,13090.476830,13185.253280,13188.029620,13438.282890,13686.882360
241,Zambia,ZMB,1517.926292,1528.781247,1499.340446,1600.347799,1457.627696,1493.123972,1572.973238,1616.492883,...,3017.024571,3269.794329,3419.010460,3634.780353,3765.584162,3893.549478,3927.761816,3998.004435,4090.120040,4223.906936
